In [1]:
import numpy as np
import scipy as sp
from scipy import linalg
from scipy import optimize
from scipy import interpolate
import sympy as sm
from scipy.special import erfinv
import pandas as pd
import seaborn as sns
%matplotlib inline
import matplotlib.pyplot as plt

from sklearn.linear_model import LogisticRegression
import statsmodels.api as sm
from statsmodels.discrete.discrete_model import Logit, Probit, MNLogit
import statsmodels.formula.api as smf
import warnings
warnings.filterwarnings('ignore')

In [2]:
#importing all datasets(!!please change the path before running the codes)
df_Calls = pd.read_csv(r'b. CARTIER_CALLS.csv', encoding='latin-1')
df_Clienteling = pd.read_csv(r'c. CARTIER_CLIENTELING.csv')
df_Livechat = pd.read_csv(r'd. CARTIER_LIVECHAT.csv')
df_PrevSales = pd.read_csv(r'e. CARTIER_PREVIOUS_SALES.csv')
df_Sales = pd.read_csv(r'f. CARTIER_SALES.csv')
df_Wishlist = pd.read_csv(r'g. CARTIER_WISHLIST.csv')

#concat purchase and repurchase and generate an interaction column
df_Sales['in_salesdataset']=1
df_PrevSales['in_salesdataset']=0
df_AllSales = pd.concat([df_Sales, df_PrevSales])
df_AllSales = df_AllSales.sort_values(by='ClientID', ascending= True)

df_AllSales['Calls']=np.where(df_AllSales['ClientID'].isin(df_Calls['ClientID'])==True,'yes','no')
df_AllSales['LiveChat']=np.where(df_AllSales['ClientID'].isin(df_Livechat['cLientid'])==True,'yes','no')
df_AllSales['Clienteling']=np.where(df_AllSales['ClientID'].isin(df_Clienteling['clientID'])==True,'yes','no')
df_AllSales['Wishlist']=np.where(df_AllSales['ClientID'].isin(df_Wishlist['clientID'])==True,'yes','no')

In [3]:
#Only leave "Sale" for the transaction category
data1=df_AllSales.drop(df_AllSales[(df_AllSales['TransactionCategory']!='Sale')].index)
#data1=data1.dropna(subset=['nb_days_since_last_sale'])

In [4]:
# Remove duplicates
data1.drop_duplicates(inplace=True)
data1.shape[0]

1717773

In [5]:
#View the data if needed
#data1sample=data1.head(int(round((len(df_AllSales)/(50)),0)))
#data1sample.to_csv('df_DATA1.csv')

#Collect all the needed raw data 
data2=data1[['ClientID','Gender','Channel','AgeAtTransaction','nb_days_since_last_sale',
             'TransactionDate','WeddingDate','PersonBirthDate','Calls','LiveChat','Clienteling','Wishlist', 'Turnover']]
data2=pd.DataFrame(data2)

In [6]:
data2.sort_values(['ClientID', 'TransactionDate'], ascending = [True,False], inplace=True)
data2.head()

,ClientID,Gender,Channel,AgeAtTransaction,nb_days_since_last_sale,TransactionDate,WeddingDate,PersonBirthDate,Calls,LiveChat,Clienteling,Wishlist,Turnover
22358,0011i00000UNT9LAAX,Male,Boutique,59.0,1016.0,2022-09-25,NaN,1964-10-13,no,no,no,no,8205.120000
122626,0011i00000UNT9LAAX,Male,Boutique,56.0,2441.0,2019-12-14,NaN,1964-10-13,no,no,no,no,65.678613
134321,0011i00000UNT9LAAX,Male,Boutique,56.0,2441.0,2019-12-14,NaN,1964-10-13,no,no,no,no,81.143832
125277,0011i00000UNT9LAAX,Male,Boutique,56.0,2441.0,2019-12-14,NaN,1964-10-13,no,no,no,no,479.618832
60779,0011i00000UNTHbAAP,Male,Boutique,29.0,337.0,2022-05-18,2019-06-15,1994-11-23,no,no,yes,no,220.830000


In [7]:
data2

,ClientID,Gender,Channel,AgeAtTransaction,nb_days_since_last_sale,TransactionDate,WeddingDate,PersonBirthDate,Calls,LiveChat,Clienteling,Wishlist,Turnover
22358,0011i00000UNT9LAAX,Male,Boutique,59.0,1016.0,2022-09-25,NaN,1964-10-13,no,no,no,no,8205.120000
122626,0011i00000UNT9LAAX,Male,Boutique,56.0,2441.0,2019-12-14,NaN,1964-10-13,no,no,no,no,65.678613
134321,0011i00000UNT9LAAX,Male,Boutique,56.0,2441.0,2019-12-14,NaN,1964-10-13,no,no,no,no,81.143832
125277,0011i00000UNT9LAAX,Male,Boutique,56.0,2441.0,2019-12-14,NaN,1964-10-13,no,no,no,no,479.618832
60779,0011i00000UNTHbAAP,Male,Boutique,29.0,337.0,2022-05-18,2019-06-15,1994-11-23,no,no,yes,no,220.830000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1468692,0011i00001VcMuUAAV,Male,Wholesale,NaN,NaN,2022-06-22,NaN,NaN,no,no,no,no,6367.515000
17119,0011i00001VcgKDAAZ,Female,Wholesale,42.0,788.0,2022-09-18,NaN,1981-04-08,no,no,no,no,5525.065454
92232,0011i00001VcgKDAAZ,Female,Wholesale,40.0,NaN,2020-07-22,NaN,1981-04-08,no,no,no,no,4708.727576
252156,0011i00001Vdw67AAB,Male,Boutique,42.0,682.0,2021-07-07,NaN,1980-12-20,no,no,no,no,273.295782


In [8]:
#dfN = data2.groupby('ClientID').apply(lambda x:x['TransactionDate'].reset_index()).reset_index()
Data3 = data2.groupby('ClientID').head(1)
Data3.sort_values(['TransactionDate'], ascending = False, inplace=True)
Data3.head(10)

,ClientID,Gender,Channel,AgeAtTransaction,nb_days_since_last_sale,TransactionDate,WeddingDate,PersonBirthDate,Calls,LiveChat,Clienteling,Wishlist,Turnover
258869,0011i00000VnOzJAAV,Female,Boutique,NaN,670.0,2022-09-27,NaN,NaN,no,no,yes,no,328.090000
218195,0011i00000VmrtxAAB,Male,Boutique,37.0,2140.0,2022-09-27,NaN,1986-04-20,no,no,no,no,2502.640000
507436,0011i00000VqDZbAAN,Male,Boutique,NaN,285.0,2022-09-27,NaN,NaN,no,no,yes,no,12069.271340
391204,0011i00000uRLEzAAO,Male,Boutique,NaN,651.0,2022-09-26,NaN,NaN,no,no,no,no,5384.610000
270306,0011i00000VulJMAAZ,Female,Boutique,56.0,241.0,2022-09-26,NaN,1967-05-23,no,no,yes,no,9487.170000
408405,0011i00000rVmzUAAS,Male,Boutique,50.0,502.0,2022-09-26,NaN,1973-10-07,no,no,no,no,34392.640202
399347,0011i00001TsKmUAAV,Female,Boutique,28.0,71.0,2022-09-26,NaN,1995-07-01,no,no,yes,no,8034.180000
327768,0011i00000VuheKAAR,Male,Boutique,34.0,2327.0,2022-09-26,NaN,1989-05-05,no,no,no,no,1426.810000
487272,0011i00000rVnlTAAS,Male,Boutique,NaN,824.0,2022-09-26,NaN,NaN,no,no,no,no,6013.091440
516682,0011i00000VlyxUAAR,Male,Boutique,56.0,4264.0,2022-09-26,2010-12-27,1967-02-13,no,no,no,no,907.970000


In [10]:
Data3.isnull().mean()

ClientID                   0.000000
Gender                     0.000873
Channel                    0.000000
AgeAtTransaction           0.373191
nb_days_since_last_sale    0.079715
TransactionDate            0.000000
WeddingDate                0.891615
PersonBirthDate            0.312407
Calls                      0.000000
LiveChat                   0.000000
Clienteling                0.000000
Wishlist                   0.000000
Turnover                   0.021422
dtype: float64

In [11]:

#Pre-process the dependent variable "repurchase"
Data3['repurchase'] = np.where(Data3['nb_days_since_last_sale']>=1460,'yes','no')
print(Data3['repurchase'].value_counts())
#Making"repurchase" into a dummy dependent variable
dummy_repurchase=pd.get_dummies(Data3['repurchase'],prefix='repurchase')
Data3['repurchase_yes']=dummy_repurchase['repurchase_yes']
#here repurchase is 1 and not repurchase is 0.
print(Data3['repurchase_yes'].value_counts())

no     298192
yes     47853
Name: repurchase, dtype: int64
0    298192
1     47853
Name: repurchase_yes, dtype: int64


In [12]:
#Pre-process "Gender" as the first dummy independent variable
dummy_Gender=pd.get_dummies(Data3['Gender'],prefix='Gender')
Data3['Gender_Female']=dummy_Gender['Gender_Female']
#here female is 1 and male is 0.

#Pre-process "AgeAtTransaction" as a serial independent variable
Data3= Data3.dropna(subset=['AgeAtTransaction'])
Data3['AgeAtTransaction']=Data3['AgeAtTransaction'].astype(int)
Data3.head()

,ClientID,Gender,Channel,AgeAtTransaction,nb_days_since_last_sale,TransactionDate,WeddingDate,PersonBirthDate,Calls,LiveChat,Clienteling,Wishlist,Turnover,repurchase,repurchase_yes,Gender_Female
218195,0011i00000VmrtxAAB,Male,Boutique,37,2140.0,2022-09-27,NaN,1986-04-20,no,no,no,no,2502.640000,yes,1,0
270306,0011i00000VulJMAAZ,Female,Boutique,56,241.0,2022-09-26,NaN,1967-05-23,no,no,yes,no,9487.170000,no,0,1
408405,0011i00000rVmzUAAS,Male,Boutique,50,502.0,2022-09-26,NaN,1973-10-07,no,no,no,no,34392.640202,no,0,0
399347,0011i00001TsKmUAAV,Female,Boutique,28,71.0,2022-09-26,NaN,1995-07-01,no,no,yes,no,8034.180000,no,0,1
327768,0011i00000VuheKAAR,Male,Boutique,34,2327.0,2022-09-26,NaN,1989-05-05,no,no,no,no,1426.810000,yes,1,0


In [13]:
#Pre-process "Channel" as the second dummy indepedent variable.
dummy_Channel=pd.get_dummies(Data3['Channel'],prefix='Channel')
Data3['Channel_Boutique']=dummy_Channel['Channel_Boutique']
Data3['Channel_Web']=dummy_Channel['Channel_Web']
Data3['Channel_CC']=dummy_Channel['Channel_CC']
Data3.head()

,ClientID,Gender,Channel,AgeAtTransaction,nb_days_since_last_sale,TransactionDate,WeddingDate,PersonBirthDate,Calls,LiveChat,Clienteling,Wishlist,Turnover,repurchase,repurchase_yes,Gender_Female,Channel_Boutique,Channel_Web,Channel_CC
218195,0011i00000VmrtxAAB,Male,Boutique,37,2140.0,2022-09-27,NaN,1986-04-20,no,no,no,no,2502.640000,yes,1,0,1,0,0
270306,0011i00000VulJMAAZ,Female,Boutique,56,241.0,2022-09-26,NaN,1967-05-23,no,no,yes,no,9487.170000,no,0,1,1,0,0
408405,0011i00000rVmzUAAS,Male,Boutique,50,502.0,2022-09-26,NaN,1973-10-07,no,no,no,no,34392.640202,no,0,0,1,0,0
399347,0011i00001TsKmUAAV,Female,Boutique,28,71.0,2022-09-26,NaN,1995-07-01,no,no,yes,no,8034.180000,no,0,1,1,0,0
327768,0011i00000VuheKAAR,Male,Boutique,34,2327.0,2022-09-26,NaN,1989-05-05,no,no,no,no,1426.810000,yes,1,0,1,0,0


In [14]:
#Make variables calls, livechat, clienteling, and wishlist into dummy variables.
dummy_Calls=pd.get_dummies(Data3['Calls'],prefix='Calls')
Data3['Calls_yes']=dummy_Calls['Calls_yes']
dummy_LiveChat=pd.get_dummies(Data3['LiveChat'],prefix='LiveChat')
Data3['LiveChat_yes']=dummy_LiveChat['LiveChat_yes']
dummy_Wishlist=pd.get_dummies(Data3['Wishlist'],prefix='Wishlist')
Data3['Wishlist_yes']=dummy_Wishlist['Wishlist_yes']
dummy_Clienteling=pd.get_dummies(Data3['Clienteling'],prefix='Clienteling')
Data3['Clienteling_yes']=dummy_Clienteling['Clienteling_yes']
#Here 1 means the interaction via calls, livechat or clitenteling exists before the transaction, or the creation of wishlist exists.

In [16]:
#Process the interval of days between Transaction Date and Birthday Date, also those betwee Transaction Date and Wedding Date
Data3['TransactionDate']=pd.to_datetime(Data3['TransactionDate'])
Data3['PersonBirthDate']=pd.to_datetime(Data3['PersonBirthDate'])
Data3['WeddingDate']=pd.to_datetime(Data3['WeddingDate'],errors = 'coerce')

#Delete unreasonable dates
Data3.loc[Data3['PersonBirthDate'].dt.year <= 1900, 'PersonBirthDate']=np.nan
Data3.loc[Data3['WeddingDate'].dt.year >= 2030, 'WeddingDate']=np.nan
Data3.loc[Data3['WeddingDate'].dt.year <= 1900, 'WeddingDate']=np.nan

In [17]:
#Recode all variables to calculate date intervals for Logit model (Might need to make it an integer)
# Wedding Date Variable
Data3['WeddingYear'] = Data3['WeddingDate'].dt.year
Data3['WeddingMonth'] = Data3['WeddingDate'].dt.month
Data3['WeddingDay'] = Data3['WeddingDate'].dt.day
# Transaction Date Var
Data3['TrxYear'] = Data3['TransactionDate'].dt.year
Data3['TrxMonth'] = Data3['TransactionDate'].dt.month
Data3['TrxDay'] = Data3['TransactionDate'].dt.day
# PersonBirth Date Var
Data3['BirthYear'] = Data3['PersonBirthDate'].dt.year
Data3['BirthMonth'] = Data3['PersonBirthDate'].dt.month
Data3['BirthDay'] = Data3['PersonBirthDate'].dt.day
Data3.head()

,ClientID,Gender,Channel,AgeAtTransaction,nb_days_since_last_sale,TransactionDate,WeddingDate,PersonBirthDate,Calls,LiveChat,...,Clienteling_yes,WeddingYear,WeddingMonth,WeddingDay,TrxYear,TrxMonth,TrxDay,BirthYear,BirthMonth,BirthDay
218195,0011i00000VmrtxAAB,Male,Boutique,37,2140.0,2022-09-27,NaT,1986-04-20,no,no,...,0,NaN,NaN,NaN,2022,9,27,1986,4,20
270306,0011i00000VulJMAAZ,Female,Boutique,56,241.0,2022-09-26,NaT,1967-05-23,no,no,...,1,NaN,NaN,NaN,2022,9,26,1967,5,23
408405,0011i00000rVmzUAAS,Male,Boutique,50,502.0,2022-09-26,NaT,1973-10-07,no,no,...,0,NaN,NaN,NaN,2022,9,26,1973,10,7
399347,0011i00001TsKmUAAV,Female,Boutique,28,71.0,2022-09-26,NaT,1995-07-01,no,no,...,1,NaN,NaN,NaN,2022,9,26,1995,7,1
327768,0011i00000VuheKAAR,Male,Boutique,34,2327.0,2022-09-26,NaT,1989-05-05,no,no,...,0,NaN,NaN,NaN,2022,9,26,1989,5,5


In [19]:
#Calculate the days between birthday and transaction dates.
#We only take months and days into consideration because we test the occasion of wedding anniversay and birthday.
Data3["Month_Diff1"]=Data3[["WeddingMonth","TrxMonth"]].apply(lambda x:x["WeddingMonth"]-x["TrxMonth"],axis=1)
Data3['Wed_interval']=np.where(Data3["Month_Diff1"]<0,(abs(Data3["Month_Diff1"]))*30+(Data3['WeddingDay']-Data3['TrxDay']),(abs(Data3["Month_Diff1"]))*30+(Data3['TrxDay']-Data3['WeddingDay']))
Data3['Wed_interval']=abs(Data3['Wed_interval'])

Data3["Month_Diff2"]=Data3[["BirthMonth","TrxMonth"]].apply(lambda x:x["BirthMonth"]-x["TrxMonth"],axis=1)
Data3['Birth_interval']=np.where(Data3["Month_Diff2"]<0,(abs(Data3["Month_Diff2"]))*30+(Data3['BirthDay']-Data3['TrxDay']),(abs(Data3["Month_Diff2"]))*30+(Data3['TrxDay']-Data3['BirthDay']))
Data3['Birth_interval']=abs(Data3['Birth_interval'])

#Make the interval days between birthday and transaction dates, between wedding date and transaction date as third and fourth dummy independent variables
Data3['Occasion_Wed'] = np.where(Data3['Wed_interval']<=60,'yes','no')
Data3['Occasion_Birth'] = np.where(Data3['Birth_interval']<=60,'yes','no')

dummy_Occasion_Wed=pd.get_dummies(Data3['Occasion_Wed'],prefix='Occasion_Wed')
Data3['Occasion_Wed']=dummy_Occasion_Wed['Occasion_Wed_yes']
dummy_Occasion_Birth=pd.get_dummies(Data3['Occasion_Birth'],prefix='Occasion_Birth')
Data3['Occasion_Birth']=dummy_Occasion_Birth['Occasion_Birth_yes']
#here is purchased within 60 days before birthday or wedding is 1, otherwise0.

In [87]:
# WE can see mistakes in the code 
#print(Data3[['TransactionDate', 'PersonBirthDate', 'Occasion_Birth']].head(50))
#print(data2[['TransactionDate', 'WeddingDate', 'Occasion_Wed']].head(50))

#ClientData = data2.groupby(['ClientID'])['Gender_Female'],'AgeAtTransaction','Channel_Boutique','Channel_Web','Channel_CC',
 #                       'Calls_yes','Clienteling_yes','LiveChat_yes','Wishlist_yes','Occasion_Wed','Occasion_Birth','intercept']]
    
#ClientDataTest = Data3.groupby(['ClientID'])[['repurchase_yes','Gender_Female','Channel_Boutique','AgeAtTransaction','Channel_Web','Channel_CC',
           #             'Calls_yes','Clienteling_yes','LiveChat_yes','Wishlist_yes','Occasion_Wed','Occasion_Birth', 'Turnover']].max()    

In [57]:
#add constant
#Data3['intercept']=1.0
Model1=Data3[['repurchase_yes','Gender_Female','Channel_Boutique','AgeAtTransaction','Channel_Web','Channel_CC',
                        'Calls_yes','Clienteling_yes','LiveChat_yes','Wishlist_yes','Occasion_Wed','Occasion_Birth','Turnover']]
Model2=Data3.copy()

In [60]:
#Since there are too many missing values in wedding date and birth date, we build up two models, one with them and one without.
#extract Indepedent variables into a dataframe for the model without wedding date
Model1=Model1.dropna()
Inde_var1=Model1.loc[:,['Gender_Female','Channel_Boutique','AgeAtTransaction','Channel_Web','Channel_CC',
                        'Calls_yes','Clienteling_yes','Wishlist_yes','Occasion_Wed','Occasion_Birth', 'Turnover']]
#fit the logit model
logit1=sm.Logit(Model1['repurchase_yes'],Inde_var1)
result1=logit1.fit()
print(result1.summary())

Optimization terminated successfully.
         Current function value: inf
         Iterations 7
                           Logit Regression Results                           
Dep. Variable:         repurchase_yes   No. Observations:               213922
Model:                          Logit   Df Residuals:                   213911
Method:                           MLE   Df Model:                           10
Date:                Wed, 04 Jan 2023   Pseudo R-squ.:                     inf
Time:                        14:28:03   Log-Likelihood:                   -inf
converged:                       True   LL-Null:                        0.0000
Covariance Type:            nonrobust   LLR p-value:                     1.000
                       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------
Gender_Female       -0.4317      0.013    -34.047      0.000      -0.457      -0.407
Channel_Boutique

In [118]:
### Run Sample run with regular regression and check VIF (Multi-COL)

Model3=Data3[['nb_days_since_last_sale','Gender_Female','Channel_Boutique','AgeAtTransaction','Channel_Web','Channel_CC',
                        'Calls_yes','Clienteling_yes','LiveChat_yes','Wishlist_yes','Occasion_Wed','Occasion_Birth','Turnover']]
Model3=Model3.dropna()
Inde_var1=Model3.loc[:,['Gender_Female','Channel_Boutique','AgeAtTransaction','Channel_Web','Channel_CC',
                        'Calls_yes','Clienteling_yes','LiveChat_yes','Wishlist_yes','Occasion_Wed','Occasion_Birth','Turnover']]
Inde_var1 = sm.add_constant(Inde_var1)
#fit the Regression model
reg=sm.OLS(Model3['nb_days_since_last_sale'],Inde_var1)
result1=reg.fit()
print(result1.summary())

                               OLS Regression Results                              
Dep. Variable:     nb_days_since_last_sale   R-squared:                       0.032
Model:                                 OLS   Adj. R-squared:                  0.032
Method:                      Least Squares   F-statistic:                     539.6
Date:                     Wed, 04 Jan 2023   Prob (F-statistic):               0.00
Time:                             17:18:49   Log-Likelihood:            -1.6018e+06
No. Observations:                   195464   AIC:                         3.204e+06
Df Residuals:                       195451   BIC:                         3.204e+06
Df Model:                               12                                         
Covariance Type:                 nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------

In [119]:
# Multi-collinearity test
from statsmodels.stats.outliers_influence import variance_inflation_factor
from patsy import dmatrices
vif = pd.DataFrame()
vif["VIF Factor"]= [variance_inflation_factor(X.values, i) for i in range (X.shape[1])]
vif["variables"]= X.columns

# A VIF > 10 suggests multicollinearity problem is present
vif.round(3)

,VIF Factor,variables
0,0.000,const
1,inf,nb_days_since_last_sale
2,1.047,Gender_Female
3,3.337,Channel_Boutique
4,1.080,AgeAtTransaction
5,2.994,Channel_Web
6,1.443,Channel_CC
7,1.089,Calls_yes
8,1.149,Clienteling_yes
9,1.005,LiveChat_yes


In [117]:
# Maybe try scaling the numbers (NB: Turnover has negative int. values)

from sklearn.preprocessing import scale
m_cols = ["Turnover", "nb_days_since_last_sale"]
for col in m_cols:
    scaled_col_name = col+"scaled"
    X[scaled_col_name] = scale(X[col], with_std=True)

In [113]:
# Scaled Results
X = sm.add_constant(X)
Inde_var1=X.loc[:,['Gender_Female','Channel_Boutique','AgeAtTransaction','Channel_Web','Channel_CC',
                        'Calls_yes','Clienteling_yes','LiveChat_yes','Wishlist_yes','Occasion_Wed','Occasion_Birth','Turnoverscaled','const']]
#fit the logit model
reg=sm.OLS(X['nb_days_since_last_salescaled'],Inde_var1)
result1=reg.fit()
print(result1.summary())

                                  OLS Regression Results                                 
Dep. Variable:     nb_days_since_last_salescaled   R-squared:                       0.032
Model:                                       OLS   Adj. R-squared:                  0.032
Method:                            Least Squares   F-statistic:                     539.6
Date:                           Wed, 04 Jan 2023   Prob (F-statistic):               0.00
Time:                                   17:16:13   Log-Likelihood:            -2.7417e+05
No. Observations:                         195464   AIC:                         5.484e+05
Df Residuals:                             195451   BIC:                         5.485e+05
Df Model:                                     12                                         
Covariance Type:                       nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
---------------